# Part 1: Double Lasso and DAGs 

## 1.

In [1]:
#cargando paquetes:
using Pkg
using CSV
using Distributions
using DataFrames
using Dates
using Plots
using Random
using LinearAlgebra
using LaTeXStrings
using Lasso
using Statistics
using GLMNet
using MLJ
using StatsBase
using CSV
using StatsModels
using GLM
using CategoricalArrays


## Cargamos la data

In [4]:
df_csv = CSV.read("../../../../data/wage2015_subsample_inference.csv", DataFrame)

first(df_csv, 2)


Row,rownames,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Int64
1,10,9.61538,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,12,48.0769,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9


In [5]:
#filtramos y nos quedamos con las variables que nos contengan wage y que nos interesen
#para que no tenga errores de multicolineadlidad
df_no_wage = select(df_csv, Not([:wage, :lwage, :rownames, :occ, :ind, :shs, :hsg, :scl, :ad, :mw]))

first(df_no_wage, 5)

Row,sex,clg,so,we,ne,exp1,exp2,exp3,exp4,occ2,ind2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64
1,1.0,1.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,11,18
2,0.0,1.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,10,9
3,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,19,4
4,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,1,12
5,1.0,1.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,6,22


### convertimos las dummies

In [12]:
#convertimos a occ2 en dummie:
first_model = df_no_wage
for i in 1:22
    col_name = Symbol("occ2_$i")
    first_model = DataFrames.transform(first_model, :occ2 => ByRow(x -> x == i ? 1 : 0) => col_name)
end


#convertimos ind2 en dummie:
for i in 2:22
    col_name = Symbol("ind2_$i")
    first_model = DataFrames.transform(first_model, :ind2 => ByRow(x -> x == i ? 1 : 0) => col_name)
end

first(first_model, 5)

Row,sex,clg,so,we,ne,exp1,exp2,exp3,exp4,occ2,ind2,occ2_1,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_2,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,ind2_22
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1.0,1.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,11,18,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0.0,1.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,10,9,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,19,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,1,12,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5,1.0,1.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,6,22,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [13]:
#eliminando variables que pueden causar multicolinealidad. tambien occ2 y ind2
first_model = select(first_model, Not([:occ2, :ind2, :occ2_1, :ind2_2]))

first(first_model, 5)

Row,sex,clg,so,we,ne,exp1,exp2,exp3,exp4,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,ind2_22
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1.0,1.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0.0,1.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5,1.0,1.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


### Construimos el W

In [ ]:
#Control W

W_model = first_model

# Variables que nos interesan
college_vars = vcat(Symbol.("clg"))

#exp_vars = Symbol.("exp", 1:4)
other_vars = vcat(Symbol.("sex"), Symbol.("so"), Symbol.("we"), Symbol.("ne"), Symbol.("exp", 1:4), Symbol.("occ2_", 2:22), Symbol.("ind2_", 3:22))

# Usar un bucle for para crear las nuevas columnas
for college_var in college_vars
    for other_var in other_vars
        col_name = Symbol("$(college_var)_$(other_var)")
        W_model = DataFrames.transform(W_model, [college_var, other_var] => ((x, y) -> x .* y) => col_name)
    end
end


W=W_model # este modelo sería  clg+sex+so+we+ne+exp1+exp2+exp3+exp4+c(occ2)+c(ind2)+(clg)*[sex+so+we+ne+exp1+exp2+exp3+exp4]

W=select(W, Not([:clg])) #quitamos el clg

Row,sex,so,we,ne,exp1,exp2,exp3,exp4,occ2_2,occ2_3,occ2_4,occ2_5,occ2_6,occ2_7,occ2_8,occ2_9,occ2_10,occ2_11,occ2_12,occ2_13,occ2_14,occ2_15,occ2_16,occ2_17,occ2_18,occ2_19,occ2_20,occ2_21,occ2_22,ind2_3,ind2_4,ind2_5,ind2_6,ind2_7,ind2_8,ind2_9,ind2_10,ind2_11,ind2_12,ind2_13,ind2_14,ind2_15,ind2_16,ind2_17,ind2_18,ind2_19,ind2_20,ind2_21,ind2_22,clg_sex,clg_so,clg_we,clg_ne,clg_exp1,clg_exp2,clg_exp3,clg_exp4,clg_occ2_2,clg_occ2_3,clg_occ2_4,clg_occ2_5,clg_occ2_6,clg_occ2_7,clg_occ2_8,clg_occ2_9,clg_occ2_10,clg_occ2_11,clg_occ2_12,clg_occ2_13,clg_occ2_14,clg_occ2_15,clg_occ2_16,clg_occ2_17,clg_occ2_18,clg_occ2_19,clg_occ2_20,clg_occ2_21,clg_occ2_22,clg_ind2_3,clg_ind2_4,clg_ind2_5,clg_ind2_6,clg_ind2_7,clg_ind2_8,clg_ind2_9,clg_ind2_10,clg_ind2_11,clg_ind2_12,clg_ind2_13,clg_ind2_14,clg_ind2_15,clg_ind2_16,clg_ind2_17,clg_ind2_18,clg_ind2_19,clg_ind2_20,clg_ind2_21,clg_ind2_22
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,1.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,0.0,0.0,1.0,42.0,17.64,74.088,311.17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,1.0,37.0,13.69,50.653,187.416,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0

### Construimos el Y

In [59]:
Y = select(df_csv, [:lwage])

Row,lwage
,Float64
1,2.26336
2,3.8728
3,2.40313
4,2.63493
5,3.36198
6,2.46222
7,2.95651
8,2.95651
9,2.48491


### Construimos el D

In [60]:
D = select(first_model, [:clg])

Row,clg
,Float64
1,1.0
2,1.0
3,0.0
4,0.0
5,1.0
6,1.0
7,0.0
8,0.0
9,0.0


In [61]:
Pkg.add("HDMjl")
using HDMjl

   Resolving package versions...
  No Changes to `C:\Users\abrah\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\abrah\.julia\environments\v1.10\Manifest.toml`


###  Aplicamos el double Lasso con lambda teórico

In [62]:
#Hacemos Lasso Reg  de Y ~ W
hdm_lasso = rlasso(Array(W), Array(Y), intercept=false)

Dict{String, Any} with 15 entries:
  "tss"          => 1675.17
  "dev"          => [-0.707422; 0.902016; … ; 0.524721; -0.119636;;]
  "model"        => [1.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.…
  "loadings"     => [0.351224; 0.328667; … ; 0.0458046; 0.0662147;;]
  "sigma"        => 0.57164
  "lambda0"      => 607.422
  "lambda"       => [213.341; 199.639; … ; 27.8227; 40.2202;;]
  "intercept"    => nothing
  "iter"         => 4
  "residuals"    => [-0.588282; 0.490697; … ; 0.28452; -0.692739;;]
  "rss"          => 1685.27
  "index"        => Bool[1, 1, 1, 1, 1, 0, 0, 1, 1, 1  …  1, 0, 0, 0, 0, 1, 0, …
  "beta"         => [-0.0364319, 0.10173, 0.164112, 0.104357, 0.018425, 0.0, 0.…
  "options"      => Dict{String, Any}("intercept"=>false, "post"=>true, "meanx"…
  "coefficients" => [-0.0364319, 0.10173, 0.164112, 0.104357, 0.018425, 0.0, 0.…

In [63]:
Ŷ = Array(W) * hdm_lasso["coefficients"];
Y̆ = Array(Y) - Ŷ

5150×1 Matrix{Float64}:
 -0.5882817953823363
  0.4906968963062681
 -0.7310070741700465
 -0.5442661406224771
  0.07084139664104327
 -0.8763636866226325
  0.042052571535247996
  0.17922205264999347
 -0.4450705944225155
 -0.29784502117190126
  ⋮
  0.31472540245076264
 -0.11454188309613933
 -0.045585941105538375
 -0.41443114765962585
 -0.281206250562263
  0.5608095462174831
  0.5128582755277238
  0.2845203130182301
 -0.6927387931464222

In [64]:
#Hacemos Lasso Reg  de D ~ w
hdm_lasso_2 = rlasso(Array(W), Array(D), intercept=false)

Dict{String, Any} with 15 entries:
  "tss"          => 1116.29
  "dev"          => [0.68233; 0.68233; … ; -0.31767; -0.31767;;]
  "model"        => [1.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.…
  "loadings"     => [0.0307325; 0.0396385; … ; 0.00250828; 0.00264417;;]
  "sigma"        => 0.0524739
  "lambda0"      => 607.422
  "lambda"       => [18.6676; 24.0773; … ; 1.52358; 1.60612;;]
  "intercept"    => nothing
  "iter"         => 4
  "residuals"    => [-0.00389586; -0.0141103; … ; 0.0; 0.0;;]
  "rss"          => 14.217
  "index"        => Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  1, 1, 1, 1, 1, 1, 1, …
  "beta"         => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.946…
  "options"      => Dict{String, Any}("intercept"=>false, "post"=>true, "meanx"…
  "coefficients" => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.946…

In [65]:
D̂ = Array(W) * hdm_lasso_2["coefficients"];
D̆ = Array(D) - D̂

5150×1 Matrix{Float64}:
 -0.0038958554576891125
 -0.014110328141523576
  0.0
  0.0
 -0.013129875590590645
  0.0015481060257910295
  0.0
  0.0
  0.0
 -0.0009956901496583903
  ⋮
  0.0
  0.0
 -0.01070334834248099
  0.0006656490659427972
  0.0047504691854273196
  0.0
  0.0
  0.0
  0.0

In [66]:
#haciendo la OLS
Y = vec(Y̆)
D = vec(D̆)
# Crear un DataFrame con los vectores 
data = DataFrame(Y=Y, D=D)
formula = @formula(Y ~ 0+D)
modelo_ols = lm(formula, data)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

Y ~ 0 + D

Coefficients:
─────────────────────────────────────────────────────────────
     Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────
D  3.16406    0.145181  21.79    <1e-99    2.87944    3.44868
─────────────────────────────────────────────────────────────

### Aplicamos el double Lasso y vemos su efecto con rlassoEffect

In [67]:
lasso_effect = rlassoEffect(Array(W), Array(Y), Array(D), method="double selection");

lasso_effect

Dict{String, Any} with 10 entries:
  "alpha"            => 3.11188
  "t"                => 19.8395
  "se"               => 0.156853
  "no_select"        => 0
  "coefficients_reg" => [0.0551397, 3.11188, -0.0013361, -0.184715, -0.255115, …
  "sample_size"      => 5150
  "coefficient"      => 3.11188
  "selection_index"  => Bool[0, 0, 0, 0, 1, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0,…
  "residuals"        => Dict("v"=>[-0.00474186, -0.011463, -0.00168841, 0.00064…
  "coefficients"     => 3.11188

In [68]:
lasso_effect["coefficients_reg"]

6-element Vector{Float64}:
  0.05513966290146513
  3.1118818110848663
 -0.0013361027088870504
 -0.18471498775349904
 -0.25511534596169627
 -0.10356177349310736

In [69]:
lasso_effect["selection_index"]

98-element BitVector:
 0
 0
 0
 0
 1
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [70]:
names(W)

98-element Vector{String}:
 "sex"
 "so"
 "we"
 "ne"
 "exp1"
 "exp2"
 "exp3"
 "exp4"
 "occ2_2"
 "occ2_3"
 ⋮
 "clg_ind2_14"
 "clg_ind2_15"
 "clg_ind2_16"
 "clg_ind2_17"
 "clg_ind2_18"
 "clg_ind2_19"
 "clg_ind2_20"
 "clg_ind2_21"
 "clg_ind2_22"

### vemos las variables que se permanecieron luego del double lasso

In [ ]:
# Nombres de las columnas
nombres_cols = names(W)

# Vector con valores
selection = lasso_effect["selection_index"]

# Filtrar los nombres que coinciden con el valor 1
nombres_filtrados = [nombres_cols[i] for i in 1:length(selection) if selection[i] == 1]

# Mostrar los nombres filtrados
println("variables que permanecieron luego del double lasso: ", nombres_filtrados)

length(nombres_filtrados)


variables que permanecieron luego del double lasso: ["exp1", "occ2_11", "occ2_14", "occ2_17"]


4